In [ ]:
!pip install catboost

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score as bas
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from catboost import CatBoostClassifier
import lightgbm  as lgb
import xgboost as xgb

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Conv/Dev_data_to_be_shared.csv')
df_vald  = pd.read_csv('/content/drive/MyDrive/Conv/Validation_data_to_be_shared.csv')

<ipython-input-402-d05889b0df76>:1: DtypeWarning: Columns (5,18,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Conv/Dev_data_to_be_shared.csv')
<ipython-input-402-d05889b0df76>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vald  = pd.read_csv('/content/drive/MyDrive/Conv/Validation_data_to_be_shared.csv')


**HIGHLY SKEWED DATASET**

In [ ]:
df['Target'].value_counts()

0    98000
1     2000
Name: Target, dtype: int64

Most of the data is numeric 154/178 columns

In [ ]:
df._get_numeric_data()

,Primary key,Target,demog_1,demog_3,demog_5,demog_6,demog_7,demog_8,demog_11,txn_1,...,others_41,txn_80,txn_81,demog_39,demog_41,others_42,others_43,others_44,others_45,demog_42
0,1,0,53.0,101.0,3.0,1.0,1.0,41.0,0.0,1.0,...,0.0,1.0,1.0,1,1,NaN,0.0,NaN,NaN,1
1,2,0,55.0,101.0,2.0,1.0,1.0,41.0,0.0,0.0,...,0.0,12.0,28.0,1,1,NaN,NaN,NaN,NaN,1
2,3,0,55.0,101.0,4.0,1.0,1.0,41.0,0.0,0.0,...,0.0,5.0,48.0,1,1,NaN,NaN,NaN,NaN,1
3,4,0,53.0,101.0,4.0,1.0,1.0,41.0,0.0,0.0,...,0.0,1.0,10.0,1,1,NaN,NaN,NaN,NaN,1
4,5,0,55.0,101.0,4.0,1.0,1.0,41.0,0.0,0.0,...,0.0,NaN,NaN,1,1,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99996,1,55.0,101.0,4.0,1.0,1.0,42.0,0.0,0.0,...,0.0,15.0,4.0,1,1,NaN,NaN,NaN,NaN,1
99996,99997,1,55.0,101.0,3.0,1.0,1.0,41.0,0.0,0.0,...,0.0,2586.0,NaN,1,1,NaN,NaN,NaN,NaN,1
99997,99998,1,55.0,101.0,4.0,1.0,1.0,41.0,0.0,0.0,...,0.0,17.0,13.0,1,1,NaN,NaN,NaN,NaN,1
99998,99999,1,55.0,101.0,3.0,1.0,1.0,42.0,0.0,0.0,...,0.0,13.0,62.0,1,1,NaN,NaN,NaN,NaN,1


Changing non-numerical data to proper forms

In [ ]:
df.select_dtypes(exclude = np.number)

,account_opening_date,country_code,demog_2,income,city_tier,occupation,demog_4,demog_9,demog_10,demog_12,...,demog_17,demog_18,demog_19,demog_20,demog_21,demog_22,os,email_domain,demog_40,demog_43
0,2/18/2023,IN,3.0,100001 to 5L,Rural,Self_Employed,N,False,N,False,...,True,False,True,True,False,Y,ios,gmail,low,medium
1,2/1/2023,IN,1.0,0 to 1L,Tier 1,Salaried,N,True,N,False,...,True,False,True,True,True,Y,and,gmail,High,High
2,12/9/2022,IN,3.0,0 to 1L,Tier 2,Self_Employed,N,False,N,False,...,True,False,True,False,False,Y,and,gmail,medium,High
3,3/31/2023,IN,3.0,100001 to 5L,Tier 7,Self_Employed,N,NaN,N,False,...,True,False,True,False,False,Y,and,gmail,High,High
4,1/17/2023,IN,6.0,0 to 1L,Tier 1,Student,N,False,N,False,...,True,False,True,False,False,Y,and,gmail,low,medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,12/21/2022,IN,2,0 to 1L,Rural,Self_Employed,N,False,N,False,...,True,False,True,False,False,Y,and,gmail,High,High
99996,3/17/2023,IN,6,100001 to 5L,Rural,Student,N,NaN,N,False,...,True,False,True,True,False,Y,and,gmail,low,medium
99997,12/8/2022,IN,1,100001 to 5L,Tier 6,Salaried,N,False,N,False,...,True,False,True,False,False,Y,and,gmail,High,medium
99998,3/3/2023,IN,1,5L to 10L,Tier 1,Other,N,NaN,N,False,...,True,False,True,False,False,Y,ios,gmail,High,High


In [ ]:
print(df['demog_2'].value_counts())
df['demog_2'].replace('ZZ','14',inplace=True)
df['demog_2'] = df['demog_2'].astype(np.float64)
df_vald['demog_2'].replace('ZZ','14',inplace=True)
df_vald['demog_2'] = df['demog_2'].astype(np.float64)

1.0     36791
3.0     14691
2.0     11834
6.0      9120
1        5217
7.0      4802
4.0      4769
3        2004
2        1633
6        1274
5.0       940
10.0      916
7         691
4         655
11.0      427
17.0      202
10        142
5         124
11         55
17         23
15.0       10
9.0         7
ZZ          3
16.0        2
Name: demog_2, dtype: int64


Replacing ZZ with 0 and casting demog_2 into numerical values.

In [ ]:
df = df.replace({True: 1, False: 0})
df_vald = df_vald.replace({True: 1, False: 0})

Replacing True / False Columns with 1 / 0

Dropping columns with zero variance

In [ ]:
lv0=[]
for col in df.columns:
  if df[col].nunique() == 1:
    lv0.append(col)
print(df[lv0].nunique())
df = df.drop(lv0,axis=1)
df_vald = df_vald.drop(lv0,axis=1)

demog_7     1
demog_10    1
demog_12    1
txn_35      1
txn_36      1
txn_49      1
txn_50      1
txn_65      1
txn_70      1
txn_71      1
txn_72      1
demog_38    1
dtype: int64


In [ ]:
print(df['demog_22'].value_counts())
df['demog_22'] = df['demog_22'].replace({'Y': 1, 'N': 0})
df_vald['demog_22'] = df_vald['demog_22'].replace({'Y': 1, 'N': 0})

Y    99625
N      374
Name: demog_22, dtype: int64


Making Demog_22 from Y/N column to 1/0 column

In [ ]:
df = df.drop('account_opening_date',axis=1)
df_vald = df_vald.drop('account_opening_date',axis=1)

Dropping Account opening Date


In [ ]:
df = df.drop('Primary key',axis=1)

Dropping Primary Key from dev_data.

In [ ]:
df_mules = df.loc[df['Target']==1]
print(df_mules.isnull().sum(axis = 0).sort_values(ascending=0))
df = df.drop(['others_45','others_44','others_42','others_43'],axis=1)
df_vald = df_vald.drop(['others_45','others_44','others_42','others_43'],axis=1)

others_45    2000
others_44    1999
others_42    1997
others_43    1994
demog_9       833
             ... 
others_11       0
others_10       0
others_9        0
others_8        0
demog_43        0
Length: 164, dtype: int64


Dropping columns with all NaN values for mule accounts

In [ ]:
print(df['demog_40'].value_counts())
print(df['demog_43'].value_counts())
df['demog_40'] = df['demog_40'].replace({'low':1,'medium':2,'High':3})
df['demog_43'] = df['demog_43'].replace({'low':1,'medium':2,'High':3})
df_vald['demog_40'] = df_vald['demog_40'].replace({'low':1,'medium':2,'High':3})
df_vald['demog_43'] = df_vald['demog_43'].replace({'low':1,'medium':2,'High':3})

low       55490
High      21080
medium    19617
Name: demog_40, dtype: int64
medium    61190
High      26294
low        8811
Name: demog_43, dtype: int64


Changing High, low , medium to 3, 1, 2.

In [ ]:
print(df['income'].value_counts())
df['income'] = df['income'].replace({"100001 to 5L":3,"0 to 1L":0.5,"5L to 10L":7.5,"0":0,"10L to 25L":17.5,"25L to 50L":37.5,"50L to 3Crore":175,"3Crore to 5Crore":400,"5Crore to 10Crore":750,"20Crore to 35Crore":2750,"10Crore to 15Crore":1250,"15Crore to 20Crore":1750})
df_vald['income'] = df_vald['income'].replace({"100001 to 5L":3,"0 to 1L":0.5,"5L to 10L":7.5,"0":0,"10L to 25L":17.5,"25L to 50L":37.5,"50L to 3Crore":175,"3Crore to 5Crore":400,"5Crore to 10Crore":750,"20Crore to 35Crore":2750,"10Crore to 15Crore":1250,"15Crore to 20Crore":1750})
#print(df['income'].value_counts())

100001 to 5L          48612
0 to 1L               32783
5L to 10L              9201
0                      3816
10L to 25L             3687
25L to 50L             1162
50L to 3Crore           686
3Crore to 5Crore         37
5Crore to 10Crore         6
20Crore to 35Crore        2
10Crore to 15Crore        1
15Crore to 20Crore        1
Name: income, dtype: int64


Making income as avg as limits. 1 lakh = 1

In [ ]:
print(df['city_tier'].value_counts())
df['city_tier']  = df['city_tier'].replace({'Rural':0,'Tier 8':1,'Tier 7':2,'Tier 6':3,'Tier 5':4,'Tier 4':5,'Tier 3':6,'Tier 2':7,'Tier 1':8})
df_vald['city_tier']  = df_vald['city_tier'].replace({'Rural':0,'Tier 8':1,'Tier 7':2,'Tier 6':3,'Tier 5':4,'Tier 4':5,'Tier 3':6,'Tier 2':7,'Tier 1':8})
#df['city_tier']

Tier 1    25680
Rural     19423
Tier 8    12078
Tier 7    10389
Tier 6     8977
Tier 3     6326
Tier 5     5629
Tier 2     4944
Tier 4     2304
Name: city_tier, dtype: int64


Handling cities with Tier 1 being highest and Rural being lowest.

In [ ]:
print(df['demog_4'].value_counts())
df['demog_4'] = df['demog_4'].replace({'N':0})
df['demog_4'] = df['demog_4'].astype(str).astype(np.float64)
df_vald['demog_4'] = df_vald['demog_4'].replace({'N':0})
df_vald['demog_4'] = df_vald['demog_4'].astype(str).astype(np.float64)

N    90142
6      455
5      404
7      114
2       28
1       15
3        1
Name: demog_4, dtype: int64


Casting demog_4 into a numerical column

In [ ]:
df['country_code'] = df['country_code'].astype('str')
df['occupation'] = df['occupation'].astype('str')
df['os'] = df['os'].astype('str')
df['email_domain'] = df['email_domain'].astype('str')

CAT boost with categorical Features

In [ ]:
X = df.drop('Target',axis=1)
y = df['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42,test_size=0.2)
ls = []
for col in df.select_dtypes(exclude=np.number).columns:
  ls.append(col)
clf_c = CatBoostClassifier(cat_features=ls)
clf_c.fit(X_train,y_train)

In [ ]:
y_pred_c = clf_c.predict(X_test)
print(f1_score(y_test,y_pred_c))
print(bas(y_test,y_pred_c))
confusion_matrix(y_test,y_pred_c)

0.9006451612903225
0.9355867346938775


array([[19574,    26],
       [   51,   349]])

In [ ]:
df[ls] = df[ls].astype('category')
df = pd.get_dummies(df)
df_vald[ls] = df_vald[ls].astype('category')
df_vald = pd.get_dummies(df_vald)

ONE-HOT encoding country, occupation, os, email

In [ ]:
X = df.drop('Target',axis=1)
y = df['Target']
X_train_na, X_test_na, y_train_na, y_test_na = train_test_split(X, y, stratify=y, random_state=42,test_size=0.2)

Keeping a copy of the values with Nans

Making correlation matrix to proceed with feature selection

In [ ]:
corr_mat = df.corr()
abs(corr_mat["Target"]).sort_values()

occupation_Unemployed    0.000125
demog_16                 0.000438
country_code_LU          0.000452
country_code_SI          0.000452
country_code_DK          0.000452
                           ...   
others_25                0.344767
others_7                 0.382656
others_13                0.394471
others_16                0.418453
Target                   1.000000
Name: Target, Length: 228, dtype: float64

In [ ]:
df_ans=pd.DataFrame()
df_ans['Primary key'] = df_vald['Primary key']
df_vald = df_vald.drop('Primary key',axis=1)
df_vald_na = df_vald
df_vald = df_vald.fillna(0)

Dropping primary key from val_data and saving it elsewhere.

In [ ]:
l1=[]
for col in df_vald.columns:
  if(col not in X_train_na.columns):
    l1.append(col)
l2=[]
for col in X_train_na.columns:
  if(col not in df_vald.columns):
    l2.append(col)
df = df.drop(l2,axis=1)
df_vald = df_vald.drop(l1,axis=1)

Making sure that both training and validation data have the same set of columns.

In [ ]:
df  = df.fillna(0)
smote = SMOTE(random_state=42)
X = df.drop('Target',axis=1)
y = df['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42,test_size=0.2)
X_train,y_train = smote.fit_resample(X_train,y_train)

Creating final train and test after making columns same. Using SMOTE to make the dataset balanced.

Using Decision Tree

In [ ]:
clf_d = DecisionTreeClassifier()
clf_d.fit(X_train,y_train)


DecisionTreeClassifier()

In [ ]:
y_pred_d = clf_d.predict(X_test)

print(f1_score(y_test,y_pred_d))
print(bas(y_test,y_pred_d))
print(confusion_matrix(y_test,y_pred_d))

0.8500619578686495
0.9271173469387755


array([[19536,    64],
       [   57,   343]])

Using Random Forest

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)

RandomForestClassifier()

In [ ]:
y_pred_f = rfc.predict(X_test)
print(f1_score(y_test,y_pred_f))
print(bas(y_test,y_pred_f))
confusion_matrix(y_test,y_pred_f)

0.9093167701863354
0.9565051020408164


array([[19561,    39],
       [   34,   366]])

Using CATboost with SMOTE

In [ ]:
clf_c0 = CatBoostClassifier()
clf_c0.fit(X_train,y_train)

In [ ]:
y_pred_c0 = clf_c0.predict(X_test)
print(f1_score(y_test,y_pred_c0))
print(bas(y_test,y_pred_c0))
confusion_matrix(y_test,y_pred_c0)

0.9029850746268657
0.9527040816326531


array([[19559,    41],
       [   37,   363]])

Using CAT boost without SMOTE

In [ ]:
clf_c0_na = CatBoostClassifier()
clf_c0_na.fit(X_train_na,y_train_na)

In [ ]:
y_pred_c0_na = clf_c0_na.predict(X_test_na)
print(f1_score(y_test_na,y_pred_c0_na))
print(bas(y_test_na,y_pred_c0_na))
print(confusion_matrix(y_test_na,y_pred_c0_na))

0.9058064516129032
0.9381377551020408
[[19576    24]
 [   49   351]]


Using lightGBM classifier.

In [ ]:
clf_l = lgb.LGBMClassifier()
clf_l.fit(X_train,y_train)

[LightGBM] [Info] Number of positive: 78400, number of negative: 78400
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.291621 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 28132
[LightGBM] [Info] Number of data points in the train set: 156800, number of used features: 176
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


LGBMClassifier()

In [ ]:
y_pred_l = clf_l.predict(X_test)
print(f1_score(y_test,y_pred_l))
print(bas(y_test,y_pred_l))
confusion_matrix(y_test,y_pred_l)

0.9111389236545683
0.9541071428571428


array([[19565,    35],
       [   36,   364]])

LightGBM without SMOTE

In [ ]:
clf_l_na = lgb.LGBMClassifier(scale_pos_weight=49)
clf_l_na.fit(X_train_na,y_train_na)

[LightGBM] [Info] Number of positive: 1600, number of negative: 78400
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.173264 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10188
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 178
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.020000 -> initscore=-3.891820
[LightGBM] [Info] Start training from score -3.891820


LGBMClassifier(scale_pos_weight=49)

In [ ]:
y_pred_l_na = clf_l_na.predict(X_test_na)
print(f1_score(y_test_na,y_pred_l_na))
print(bas(y_test_na,y_pred_l_na))
print(confusion_matrix(y_test_na,y_pred_l_na))

0.8963855421686747
0.9635204081632653
[[19542    58]
 [   28   372]]


Using XGBoost.

In [ ]:
clf_x = xgb.XGBClassifier(max_depth=6,subsample=0.8,scale_pos_weight=49)
clf_x.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
y_pred_x = clf_x.predict(X_test)
print(f1_score(y_test,y_pred_x))
print(bas(y_test,y_pred_x))
confusion_matrix(y_test,y_pred_x)

0.913939393939394
0.9700255102040816


array([[19552,    48],
       [   23,   377]])

XGBoost without SMOTE

In [ ]:
clf_x_na = xgb.XGBClassifier(max_depth=6,subsample=0.8,scale_pos_weight=49)
clf_x_na.fit(X_train_na,y_train_na)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
y_pred_x_na = clf_x_na.predict(X_test_na)
print(f1_score(y_test_na,y_pred_x_na))
print(bas(y_test_na,y_pred_x_na))
confusion_matrix(y_test_na,y_pred_x_na)

0.9108910891089109
0.9589795918367348


array([[19560,    40],
       [   32,   368]])

In [ ]:
y_ans = clf_x.predict(df_vald)
y_prob = clf_x.predict_proba(df_vald)

In [ ]:
y_prob_f=[]
for prob in y_prob:
  y_prob_f.append(prob[1])

In [ ]:
dit= {"Target":y_ans}
df_ans['Target values'] = pd.DataFrame(data=dit)
dit = {"probability":y_prob_f}
df_ans['probability'] = pd.DataFrame(data=dit)

In [ ]:
df_ans.to_csv('LAMe.csv')